In [11]:
df_invtry_new = pd.read_csv('assets/data/combined_tree_data_with_header_with_derived_neighborhood.csv')

df_invtry_new.condition.value_counts()

/Library/Python/2.7/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


Fair               28211
Poor               23342
Fair               19532
Good               12634
Good               11061
Vacancy             8666
Poor                1877
Dead                 741
Very Good            738
Very                 708
Dead                 369
Stump                313
Stump Removal        222
Excellent            175
Critical             120
Open                 106
Critical              85
Unsuitable Site       35
6/15/16                1
dtype: int64

In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
   
# OLD DATA
df_invtry_new = pd.read_csv('assets/data/combined_tree_data_with_header_with_derived_neighborhood.csv')
df_rules = pd.read_excel('assets/data/sf_tree_rules.xlsx', sheetname = 0)
df_updated_tree_data = pd.read_csv('assets/data/trees.csv')

# Drop ones without a condition score
df_tree_cond = df_updated_tree_data.dropna(subset=['condition']).copy()

# Convert case of all condition scores
df_tree_cond.condition = df_tree_cond.condition.apply(lambda cond: cond.strip().lower())

# Combine similar condition scores and ignore all others
condition_map = {
    'excellent': 'good',
    'very good': 'good',
    'very': 'good',
    'good': 'good',
    'fair': 'fair',
    'poor': 'poor',
    'critial': 'poor',
}
df_tree_cond.condition = df_tree_cond.condition.map(condition_map)

# Drop the trees that were not mapped
df_tree_cond = df_tree_cond.dropna(subset=['condition'])

# Convert to neg score for poor and positive for goodnp
df_tree_cond['condition_score'] = df_tree_cond.condition.map({'good': 1, 'fair': 0, 'poor': -1})


# Create model and fit
knn = NearestNeighbors(algorithm='ball_tree').fit(df_tree_cond[['point_x', 'point_y']])
   
# Get condition scores across the city
dists, nearest_trees = knn.kneighbors(X=[[-122.443480,37.776828]],
                                           n_neighbors=25,
                                           return_distance=True)

# Keep recommendation spots that are close enough to existing trees
mean_dist = dists[0].mean()

#if mean_dist > 0.005:
#    return ['No-Result', 'Mean distance from nearest trees is {}.'.format(mean_dist)]

nearest_trees = nearest_trees[0]
df_pick = pd.DataFrame()
local_trees = df_tree_cond.iloc[nearest_trees]
df_pick['condition'] = local_trees.groupby(['botanical',
                                            'common']).mean().condition_score
df_pick['count'] = local_trees.groupby(['botanical',
                                            'common']).count().condition_score

pick = df_pick.sort(['condition', 'count'], 
                    ascending = False).reset_index()


pick['botanical'] = pick['botanical'].apply(lambda x: x.lower())
accepted_species = df_rules['species'].apply(lambda x: x.lower()).reset_index(0)

joined = pd.merge(pick, accepted_species, left_on = 'botanical', right_on = 'species', how='inner')

final_pick = joined['common'].apply(lambda x: x.lower()).tolist()

final_pick

/Library/Python/2.7/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


['tarata', 'brisbane box']

In [9]:
joined

,botanical,common,condition,count,index,species
0,pittosporum eugenioides,Tarata,1.00,1,65,pittosporum eugenioides
1,lophostemon confertus,Brisbane Box,0.75,16,10,lophostemon confertus


In [10]:
final_pick = joined['common'].apply(lambda x: x.lower()).tolist()
final_pick

['tarata', 'brisbane box']

In [11]:
pick['botanical'] = pick['botanical'].apply(lambda x: x.lower())
accepted_species = df_rules['species'].apply(lambda x: x.lower()).reset_index(0)

joined = pd.merge(pick, accepted_species, left_on = 'botanical', right_on = 'species', how='inner')

In [12]:
pick

,botanical,common,condition,count
0,pittosporum eugenioides,Tarata,1.000000,1
1,lophostemon confertus,Brisbane Box,0.750000,16
2,juniperus chinensis 'torulosa',Hollywood Juniper,0.333333,3
3,cupressocyparis leylandii,Leyland Cypress,0.000000,3
4,magnolia grandiflora,Southern Magnolia,0.000000,1
5,acacia melanoxylon,Blackwood Acacia,-1.000000,1


In [13]:
joined

,botanical,common,condition,count,index,species
0,pittosporum eugenioides,Tarata,1.00,1,65,pittosporum eugenioides
1,lophostemon confertus,Brisbane Box,0.75,16,10,lophostemon confertus


In [14]:
df_rules[df_rules['species'].str.contains('', na = False)]

,category,species,notes,size,tree_type
0,"Section 1: Tree species, varieties, and cultiv...",Arbutus x ‘Marina’,Fruit drop can range from low volume to signif...,Small - \nLess than 20’ tall at maturity,Evergreen
1,"Section 1: Tree species, varieties, and cultiv...",Ceanothus ‘Ray Hartman’,Not good for narrow sidewalks,Small - \nLess than 20’ tall at maturity,Evergreen
2,"Section 1: Tree species, varieties, and cultiv...",Laurus nobilis ‘Saratoga’,"Uneven performer, prefers heat, needs some win...",Small - \nLess than 20’ tall at maturity,Evergreen
3,"Section 1: Tree species, varieties, and cultiv...",Magnolia grandiflora ‘Little Gem’,NaN,Small - \nLess than 20’ tall at maturity,Evergreen
4,"Section 1: Tree species, varieties, and cultiv...",Prunus serrulata ‘Kwanzan’,Prefers wind protection; prefers a larger plan...,Small - \nLess than 20’ tall at maturity,Deciduous
5,"Section 1: Tree species, varieties, and cultiv...",Agonis flexuosa,"Fast grower – more than 12” annually, requires...",Medium -\n20-35’ tall at maturity,Evergreen
6,"Section 1: Tree species, varieties, and cultiv...",Callistemon viminalis,Has sticky flowers,Medium -\n20-35’ tall at maturity,Evergreen
7,"Section 1: Tree species, varieties, and cultiv...","Magnolia grandiflora ‘St. Mary,’",NaN,Medium -\n20-35’ tall at maturity,Evergreen
8,"Section 1: Tree species, varieties, and cultiv...",Tristaniopsis laurina,"Formerly known as Tristania laurina, slow grow...",Medium -\n20-35’ tall at maturity,Evergreen
9,"Section 1: Tree species, varieties, and cultiv...",Lagunaria patersonii,Grows well in windy areas,Tall – \nMore than 35’ tall at maturity,Evergreen


In [15]:
df_rules[df_rules['species'].str.contains("Magnolia grandiflora", na = False)]

,category,species,notes,size,tree_type
3,"Section 1: Tree species, varieties, and cultiv...",Magnolia grandiflora ‘Little Gem’,NaN,Small - \nLess than 20’ tall at maturity,Evergreen
7,"Section 1: Tree species, varieties, and cultiv...","Magnolia grandiflora ‘St. Mary,’",NaN,Medium -\n20-35’ tall at maturity,Evergreen
11,"Section 1: Tree species, varieties, and cultiv...","Magnolia grandiflora ‘Sam Sommers,’ ‘Majestic ...",NaN,Tall – \nMore than 35’ tall at maturity,Evergreen


In [16]:
df_rules.species.value_counts()

Pyrus calleryana ‘New Bradford’               2
Laurus nobilis ‘Saratoga’                     2
Cedrella fissilis                             1
Platanus x acerifolia ‘Liberty’               1
Olea europaea                                 1
Liriodendron tulipifera                       1
Tilia cordata                                 1
Ulmus parvifolia ‘Drake’ ‘Sempervirens’       1
Prunus serrulata ‘Kwanzan’                    1
Ulmus parvifolia x carpinifolia ‘Frontier’    1
Chorisia speciosa                             1
Acer x freemanii                              1
Acer rubrum ‘Armstrong’                       1
Brachychiton populneum                        1
Liquidambar styraciflua                       1
...
Pittosporum undulatum                     1
Pistacia chinensis ‘Keith Davey’          1
Tristaniopsis laurina                     1
Magnolia champaca                         1
Metrosideros excelsus                     1
Cassia leptophylla                        1
Quercus tome

In [17]:
df_updated_tree_data.botanical.value_counts()

Sawcut New Tree Well Site       30370
Platanus x acerifolia            6808
Ficus microcarpa 'Nitida'        5486
Pittosporum undulatum            5358
Prunus cerasifera                4971
Magnolia grandiflora             4815
Vacant Planting Site - Small     4590
Lophostemon confertus            4579
Metrosideros excelsa             4160
Tristaniopsis laurina            3778
Prunus serrulata 'Kwanzan'       3173
Acacia melanoxylon               3077
Arbutus 'Marina'                 2785
Maytenus boaria                  2753
Olea europaea                    2744
...
Pinus strobus                  1
Citrus maxima                  1
Catalpa speciosa               1
Buxus sempervirens             1
Acer pseudoplatanus            1
Agave attenuata                1
Acer campestre                 1
Citrus species                 1
Fagus sylvatica 'Pendula'      1
Torreya californica            1
Cedrus deodara 'Blue Snake'    1
Pinus contorta                 1
Holodiscus discolor           

In [18]:
df_updated_tree_data['botanical'] = df_updated_tree_data['botanical'].apply(lambda x: x.lower())
accepted_species = df_rules['species'].apply(lambda x: x.lower()).reset_index(0)

joined_all = pd.merge(accepted_species, df_updated_tree_data, left_on = 'species', right_on = 'botanical', how='left')

In [19]:
joined_all['species'].value_counts().count()

81

In [20]:
joined_all['botanical'].value_counts().count()

44

 Almost half of the species didnt get joined

In [31]:
joined_all[joined_all['botanical'].isnull() == True]['species'].value_counts()

pyrus calleryana ‘new bradford’                                2
laurus nobilis ‘saratoga’                                      2
agonis flexuosa ‘after dark’                                   1
brachychiton populneum                                         1
quercus frainetto ‘forest green’                               1
quercus phellos                                                1
lyonothamnus floribundus asplenifolius                         1
ceanothus ‘ray hartman’                                        1
ulmus wilsoniana ‘prospector’                                  1
elaeocarpus decipens                                           1
phoenix dactylifera ‘medjool’ or ‘zahidi’                      1
chorisia speciosa                                              1
magnolia grandiflora ‘little gem’                              1
platanus x acerifolia ‘bloodgood’  ‘columbia’ ‘yarwood’        1
ulmus parvifolia ‘drake’ ‘sempervirens’                        1
ulmus parvifolia x carpin

In [22]:
df_updated_tree_data[df_updated_tree_data['botanical'].str.contains("arbutus", na = False)]['botanical'].value_counts()

arbutus 'marina'            2785
arbutus x marina             691
arbutus unedo                502
arbutus unedo 'compacta'       3
arbutus menziesii              2
dtype: int64

In [23]:
df_updated_tree_data[df_updated_tree_data['botanical'].str.contains("arbutus", na = False)]['common'].value_counts()

Marina Arbutus             2785
strawberry marina tree      691
Strawberry Tree             441
strawberry tree              61
Compact Strawberry Tree       3
Pacific Madrone               2
dtype: int64

In [24]:
df_rules.category.value_counts()

Section 2: Tree species, varieties, and cultivars that do well with certain special considerations, which may not appropriate for planting broadly throughout the San Francisco.    40
Section 3: Tree species, varieties, and cultivars which are experimental. \nSan Francisco does not have many of these species planted yet and would like to plant more of these trees to test how they do.    27
Section 1: Tree species, varieties, and cultivars that do well in most locations in the San Francisco.*    16
dtype: int64

In [25]:
df_invtry_new[df_invtry_new['scientific_species_name'].str.contains('Magnolia grandiflora', na = False)]

NameError: name 'df_invtry_new' is not defined